# Importing Essential Libraries

In [1]:
import sys, os
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import bokeh.models as bokeh_models

sys.path.append(os.path.abspath('../..')) # Ideally ST_Visions will be installed as a module, therefore this is more of a development setting for quick testing.

from st_visions.st_visualizer import st_visualizer
import st_visions.express as viz_express
import st_visions.geom_helper as viz_helper

# Defining Global Variables

If the jupyter notebook is hosted locally, the default parameter of every notebook visualization function call is set as `localhost:localport`. If you are working with a remote server, you can set your server url in the configuration variable below (`NOTEBOOK_URL`) 

In [2]:
NOTEBOOK_URL='http://<NOTEBOOK_IP_ADDRESS>:<NOTEBOOK_PORT>' # add your own server address

# Visualizing a Simple Geometry Dataset

For the purpose of this notebook example, the Brest AIS Dataset provided chorochronos.datastories.org will be used.

[1] https://chorochronos.datastories.org/?q=node/9



In [3]:
st_viz = st_visualizer(limit=500) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath='..\..\data\chorochronos_ais_2009.csv', nrows=30000, delimiter=';')

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='fixed', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
circ = st_viz.add_glyph(marker='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')
tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()
st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, height=540, width=540)

Loading BokehJS ...

### (ADVANCED) Responsive Canvas

In [ ]:
st_viz = st_visualizer(limit=500) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath='..\..\data\chorochronos_ais_2009.csv', nrows=30000, delimiter=';')

# set up the canvas
st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
circ = st_viz.add_glyph(marker='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')
tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

# add the tools of the canvas
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()
st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

# Filters
The library is able to add filters of multiple types 
#### Temporal Filter

In [ ]:
st_viz = st_visualizer(limit=500) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath='..\..\data\chorochronos_ais_2009.csv', nrows=30000, delimiter=';')

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

st_viz.add_temporal_filter(temporal_name='ts', step_ms=500, title='Temporal Horizon')

st_viz.show_figures(notebook=True)

#### Numerical Filter (range)

In [ ]:
st_viz = st_visualizer(limit=500) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath='..\..\data\chorochronos_ais_2009.csv', nrows=30000, delimiter=';')

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

st_viz.add_numerical_filter(filter_mode='range', title='Speed (knots)', numeric_name='speed', step=1)
st_viz.show_figures(notebook=True)

#### Numerical Filter (>=)

In [ ]:
st_viz = st_visualizer(limit=500) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath='..\..\data\chorochronos_ais_2009.csv', nrows=30000, delimiter=';')

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

st_viz.add_numerical_filter(filter_mode='>=', title='Speed (knots) >=', numeric_name='speed', step=1)

st_viz.show_figures(notebook=True)

#### Categorical Filter

In [ ]:
glife = pd.read_csv('..\..\data\dummy_categorical_gdf.csv')
glife = viz_helper.getGeoDataFrame_v2(glife, crs='epsg:4326')
st_viz = st_visualizer(limit=500)
st_viz.set_data(glife)
tooltips = [('User ID','@user_id'), ('Vehicle','@label')]
viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)
st_viz.add_categorical_filter(title='Vehicle', categorical_name='label')
st_viz.show_figures(notebook=True, sizing_mode='stretch_both')



# A Note Towards Filter Interaction

In [10]:
world_airports = pd.read_csv(r'..\..\data\airports_dummy.csv')
world_airports = viz_helper.getGeoDataFrame_v2(world_airports, coordinate_columns=['Longitude', 'Latitude'], crs='epsg:4326')

''' For webmercator, when latitudes tend to 90 deg, northing tend to infinity, so using 89.9999 is not the solution. 
    The recommended area of use of EPSG:3857 is for latitudes between -85 and 85 degrees.
    Thus, we get the locations that reside anywhere but the poles. '''

world_airports = world_airports.loc[world_airports.Latitude.between(-85, 85)].copy()
world_airports = world_airports.loc[world_airports.Altitude >= 0].copy()

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.set_data(world_airports, sp_columns=['Longitude', 'Latitude'])

tooltips = [('Name','@Name'), ('City Country','@City_Country'), ('IATA','@IATA'), ('Location','(@Longitude, @Latitude, @Altitude)'), ('Timezone', '@Timezone.1')]
viz_express.plot_points_on_map(st_viz, size=5, tools=['hover,lasso_select'], tooltips=tooltips)

st_viz.add_categorical_filter()
st_viz.add_numerical_filter(filter_mode='>=', callback_policy='value_throttled')

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)
st_viz.show_figures(notebook=True)


# Α Note Towards (Multi)Polygon Visualization

updated for shapely 2.0 handling of multipolygons


https://shapely.readthedocs.io/en/stable/migration.html

In [2]:
url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=9135811&params=0'
saronic_gdf = gpd.read_file(url)

first_polygon = list(saronic_gdf.geometry.iloc[0].geoms)[0]

geom_final = shapely.geometry.Polygon(first_polygon.exterior.coords, [inner.exterior.coords for inner in list(saronic_gdf.geometry.iloc[0].geoms)[1:]])

saronic_gulf = saronic_gdf.copy()
saronic_gulf.loc[:, saronic_gulf.geometry.name] = geom_final

## Visualizing Simple Polygons

In [ ]:
st_viz = st_visualizer(allow_complex_geometries=False)
st_viz.set_data(saronic_gulf)

viz_express.plot_polygons_on_map(st_viz, polygon_type='patches')
st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

## Visualizing Complex Geometries

#TODO update the entire geometry logic for shapely 2.0

In [ ]:
st_viz = st_visualizer(allow_complex_geometries=True)
st_viz.set_data(saronic_gulf)

#viz_express.plot_polygons_on_map(st_viz, polygon_type='multi_polygons')
# st_viz.show_figures(notebook=True, sizing_mode='stretch_both')